In [ ]:
#Install Packages
!pip install rasterio
!pip install numpy
!pip install tqdm
!pip install shapely

In [ ]:
#import libraries
from osgeo import gdal
import numpy as np
import tempfile # for generating temporary file such as slope and aspect 
import rasterio as rio
from pathlib import Path # to set file path (likely from rasterio)
import rasterio.mask as rio_mask
from shapely.geometry import box 
from rasterio.vrt import WarpedVRT

In [ ]:
# CHANGE path for Sentinel image and DEM 
dem_path = Path("/Path/T06VXP_DEM_UTM6.tif") # DEM path
src_path = Path("/Path/S2B_T06VXP.tif") #Sentinel image path


In [ ]:
#Path to save output file
dst_path = "/Output path/DEMlayer.tif"

In [ ]:
#Clip and reproject DEM and calculate slope and aspect. Output image will have three bands (DEM, slope, and aspect) 
with rio.open(src_path, 'r') as img, rio.open(dem_path, 'r') as dem:
    with WarpedVRT(dem, crs=img.crs) as dem_vrt:
        meta = dem_vrt.meta.copy() # saving the meta-data of dem in variable meta
        bbox = box(*img.bounds)
        dst_img, dst_transform = rio_mask.mask(
          dataset=dem_vrt,
          shapes=(bbox,),
          invert=False,
          all_touched=False,
          crop=True,
          filled=True
        )
        meta['driver'] = 'GTiff'
        meta['count'], meta['height'], meta['width'] = dst_img.shape
        meta['transform'] = dst_transform
        _, dem_file = tempfile.mkstemp(text=False)
        _, slope_file = tempfile.mkstemp(text=False)
        _, aspect_file = tempfile.mkstemp(text=False)

        with rio.open(dem_file, 'w', **meta) as dst:
          dst.write(dst_img)
        ds = gdal.DEMProcessing(slope_file, dem_file, 'slope')
        ds = None
        ds = gdal.DEMProcessing(aspect_file, dem_file, 'aspect')
        ds = None
        with rio.open(dem_file, 'r') as dem, rio.open(slope_file, 'r') as slope, rio.open(aspect_file, 'r') as aspect:
          arr = np.concatenate([dem.read(), slope.read(), aspect.read()], axis=0)
        Path(dem_file).unlink()
        Path(slope_file).unlink()
        Path(aspect_file).unlink()
        meta['count'] = 3
        with rio.open(dst_path, 'w', **meta) as dst:
          dst.write(arr) 